In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install necessary libraries
!pip install onnxruntime
!pip install onnx
!pip3 install ktrain==0.26.4
!pip install tf2onnx
!pip install torch==1.5.0
!pip install transformers=='3.0.2'

In [3]:
!pip install 'h5py==2.10.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 4.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [7]:
import onnxruntime
import numpy
import onnx
import tensorflow
import ktrain
from tensorflow.python.keras.models import load_model
import tf2onnx
import time
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers.convert_graph_to_onnx import convert
from transformers import AutoModelForSequenceClassification
from pathlib import Path
from transformers import AutoTokenizer
import transformers
from scipy.special import softmax
import pandas as pd
transformers.__version__

'3.0.2'

In [5]:
# create ONNX session
def create_onnx_session(onnx_model_path, provider='CPUExecutionProvider'):
    """
    Creates ONNX inference session from provided onnx_model_path
    """

    from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers
    assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"

    # Few properties that might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = 0
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL

    # Load the model as a graph and prepare the CPU backend 
    session = InferenceSession(onnx_model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [8]:

data_of_comments = pd.read_csv('/content/drive/MyDrive/text_model_new/test_data_20220622_new.csv')

In [10]:
data_of_comments[:1000]

,email_id,form_id,label,emails_text_clean
0,220935602755053,220953569817367,non-spam,new request offer offer fire brigade unit webs...
1,220953875103052,220953569817367,non-spam,"offer fuchs fire brigade hello, wanted introdu..."
2,220998961775072,220997937489884,non-spam,prepared report date shift day week type shift...
3,221377948321056,220997937489884,non-spam,prepared report date shift day week type shift...
4,221377443822052,220997937489884,non-spam,prepared report date shift day week type shift...
...,...,...,...,...
995,212897299265069,212687536375163,non-spam,vaccine schedule form
996,221725974496065,212687536375163,non-spam,received appointment appointment thank schedul...
997,221725984695067,212687536375163,non-spam,vaccine schedule form
998,211587014357051,210063238869863,non-spam,beef mince 1kg beef mince 500g pork mince 500g...


In [12]:
docs = data_of_comments[:1000]['emails_text_clean'].values.tolist()

In [ ]:
docs

In [16]:
predictor = ktrain.load_predictor('/content/drive/MyDrive/text_model_new')
maxlen = predictor.preproc.maxlen
class_names = predictor.preproc.get_classes()

tokenizer = predictor.preproc.get_tokenizer()

In [20]:
sess = create_onnx_session("/content/drive/MyDrive/text_model_new_pt_onnx/model.onnx")
input = 'Give me money immediate this is not a spam I repeat'
tokens = tokenizer.encode_plus(input, max_length=maxlen, truncation=True)
tokens = {name: np.atleast_2d(value) for name, value in tokens.items()}
print()
print()
print("predicted class: %s" % (class_names[np.argmax(sess.run(None, tokens)[0])]))
softmax(sess.run(None,tokens)[0][0],axis=0)



predicted class: non-spam


array([0.74765193, 0.25234812], dtype=float32)

In [57]:
prob_checker = 0

def isclose(a, b, rel_tol=1e-06, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)


for i in docs:
  tokens = tokenizer(str(i), max_length=maxlen, truncation=True)
  tokens = {name: np.atleast_2d(value) for name, value in tokens.items()}
  prob_onnx = softmax(sess.run(None,tokens)[0][0],axis=0)
  prob_predictor = predictor.predict_proba(str(i))
  #For one test case checking both non spam and spam probabilities 
  if isclose(prob_onnx[0], prob_predictor[0]) and isclose(prob_onnx[1],prob_predictor[1]) == False:
    prob_checker = prob_checker + 1
print(prob_checker)


381


In [60]:
#For industry classifier
predictor = ktrain.load_predictor('/content/drive/MyDrive/predictor_industry_classifier_en')
maxlen = predictor.preproc.maxlen
class_names = predictor.preproc.get_classes()

tokenizer = predictor.preproc.get_tokenizer()

sess = create_onnx_session("/content/drive/MyDrive/predictor_industry_classifier_en_pt_onnx/model.onnx")
input = 'Give me money immediate this is not a spam I repeat'
tokens = tokenizer.encode_plus(input, max_length=maxlen, truncation=True)
tokens = {name: np.atleast_2d(value) for name, value in tokens.items()}
print()
print()
print("predicted class: %s" % (class_names[np.argmax(sess.run(None, tokens)[0])]))
softmax(sess.run(None,tokens)[0][0],axis=0)



predicted class: finance


array([9.46349872e-04, 1.17478266e-01, 3.10752948e-04, 1.58711697e-03,
       5.70726581e-02, 3.63836326e-02, 4.30284161e-03, 6.76229969e-03,
       9.60936025e-03, 7.24439695e-02, 1.19183604e-02, 6.21610496e-04,
       4.81261814e-04, 1.21628563e-03, 1.36218695e-02, 3.51311988e-03,
       4.03294802e-01, 1.52058117e-02, 2.34996304e-02, 2.26859981e-03,
       6.05523260e-03, 2.38949563e-02, 1.89840570e-02, 2.70192511e-03,
       3.87911289e-03, 9.25440714e-03, 2.63591185e-02, 1.02634041e-03,
       4.95999530e-02, 3.83295827e-02, 1.28953038e-02, 1.42912557e-02,
       9.72527545e-03, 4.64917423e-04], dtype=float32)

In [61]:
predictor.predict_proba(input)

array([9.4634912e-04, 1.1747824e-01, 3.1075210e-04, 1.5871144e-03,
       5.7072677e-02, 3.6383603e-02, 4.3028360e-03, 6.7622946e-03,
       9.6093649e-03, 7.2443932e-02, 1.1918369e-02, 6.2160945e-04,
       4.8126193e-04, 1.2162864e-03, 1.3621866e-02, 3.5131169e-03,
       4.0329477e-01, 1.5205789e-02, 2.3499614e-02, 2.2685956e-03,
       6.0552340e-03, 2.3894954e-02, 1.8984035e-02, 2.7019233e-03,
       3.8791064e-03, 9.2544071e-03, 2.6359122e-02, 1.0263387e-03,
       4.9599975e-02, 3.8329627e-02, 1.2895300e-02, 1.4291244e-02,
       9.7252885e-03, 4.6491533e-04], dtype=float32)

In [62]:
softmax(sess.run(None,tokens)[0][0],axis=0).sum()

1.0000001

In [63]:
predictor.predict_proba(input).sum()

0.9999999